# i.MX93 Key Import and Key Exchange Example

This documentation covers the key import and key exchange operations for the i.MX93 EdgeLock Secure Enclave, specifically focusing on the combined key agreement and key derivation functionality used to establish secure key import capabilities.

## 1. Prerequisites

- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the installation documentation.)
- This demo was tested with i.MX93 EVK board
- Obtain an NXP_PROD_KA_PUB key

### 1.1 NXP Production Key

The EdgeLock Secure Enclave provides a built-in production key for key agreement:

- Key ID: 0x70000000 (NXP PROD MANUFACT KEY AGREEMENT)
- Purpose: Enables consistent OEM_IMPORT_xxx keys across devices with same SRKH
- Availability: Same key across all lifecycles
- Public Key Export: Available via Public key export API (0x32)

This ensures that devices with the same Super Root Key Hash (SRKH) can use identical import keys, simplifying key management across device populations.


In [1]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2 Key Exchange Operation

### 2.1 Purpose

The key exchange operation performs a combined key agreement and key derivation to create symmetric keys stored in the EdgeLock Secure Enclave key storage. This operation is primarily used to establish OEM Master Keys for secure key import operations.

### 2.2 Key Features

- Combined Operation: Performs both key agreement (ECDH) and key derivation (HKDF) in a single operation
- Symmetric Key Creation: Only symmetric keys can be created through this operation
- Secure Storage: Derived keys are stored in EdgeLock Secure Enclave key storage
- OEM Signed Content: Input parameters are protected by OEM signed messages for integrity

### 2.3 Key Exchange Signed Message Configuration

```yaml

message:
  command:
    KEY_EXCHANGE_REQ:
      # Key storage configuration
      key_store_id: 0x454C4500
      
      # Algorithm selection
      key_exchange_algorithm: HKDF SHA256  # Options: HKDF SHA256, HKDF SHA384
      
      # Salt configuration for HKDF process
      salt_flags: 0                        # Bit 0: Salt source, Bit 1: ELE import salt
      
      # Derived key attributes
      derived_key_grp: 0                   # Group ID (0-99)
      derived_key_size_bits: 256           # Key size: 128/192/256
      derived_key_type: OEM_IMPORT_MK_SK   # Key type for import operations
      derived_key_lifetime: PERSISTENT     # VOLATILE/PERSISTENT/PERMANENT
      derived_key_usage:                   # Permission list
        - Derive
      derived_key_permitted_algorithm: HKDF SHA256
      derived_key_lifecycle: CURRENT       # Lifecycle restrictions
      derived_key_id: 0x1                  # Key identifier
      
      # ECDH configuration
      private_key_id: 0x70000000           # NXP PROD MANUFACT KEY AGREEMENT
      oem_private_key: oem_private_key.pem # OEM private key for ECDH
      nxp_prod_ka_pub: nxp.pub             # NXP production public key
```
### 2.4 OEM Master Key Generation

The key exchange operation generates an OEM_IMPORT_MK_SK (OEM Import Master Secret Key) with specific requirements:

#### Required Attributes
- Type: OEM_IMPORT_MK_SK
- Usage: Derive (Cache usage set by default)
- Permitted Algorithm: HKDF SHA256 (0x08000109)
- Size: 256 bits
- Lifecycle: User-defined
- Lifetime: User-defined (PERSISTENT recommended)

#### Derived Keys
From the OEM_IMPORT_MK_SK, two additional keys are derived:

1. OEM_IMPORT_CMAC_SK
   - Used to verify Key Import signed TLV buffers
   - Type: AES, Size: 256 bits
   - No key identifier (derived on-demand)

2. OEM_IMPORT_WRAP_SK
   - Used to decrypt wrapped keys in Key Import operations
   - Type: AES, Size: 256 bits
   - No key identifier (derived on-demand)

In [2]:
YamlDiffWidget("inputs/key_exchange.diffc").html

nxpimage signed-msg get-template -f mx93 -o workspace/key_exchange.yaml --force -m KEY_EXCHANGE_REQ
Creating workspace/key_exchange.yaml template file.


### 2.5 Generation of signed message and key derivation

The command generates the signed message for the key exchange. If the -w/--assets option is specified, also generates the assets - OEM_IMPORT_MK_SK key is derived.

```bash
nxpimage -v signed-msg export -c inputs/key_exchange.yaml
```

In [3]:
%! nxpimage -v signed-msg export -c inputs/key_exchange.yaml -w workspace/assets
 

nxpimage -v signed-msg export -c inputs/key_exchange.yaml -w workspace/assets
INFO:spsdk.image.ahab.signed_msg:Calculated input_peer_public_key_digest from OEM private key during config loading 3d164eb2e8c2c8566be458bdbdae6715f4484c8ffbacb14f248e20c3811def9b
INFO:spsdk.image.ahab.signed_msg:ECDH shared secret: 1ae9e13162f15e3b0baa0b7b959852e4d8a5c573cc0a4072e0d3a6666c728cbd
INFO:spsdk.image.ahab.signed_msg:Derived OEM_Import_MK_SK: c7e7436ed876b0176d775a658c3e9bcf59c940a41a2673b0552b7d7df504d194
INFO:spsdk.image.ahab.signed_msg:Derived OEM_Import_Wrap_SK: f2fa66b83ed82acbe4dddb5c60966cabad61cf3116b542115f10f06cf6aee825
INFO:spsdk.image.ahab.signed_msg:Derived OEM_Import_CMAC_SK: dc202a1160b4a5d11c088514e99e298b45aba12867543236fcd8774227f3e15c
INFO:spsdk.image.ahab.signed_msg:ECDH key derivation completed during configuration loading
INFO:spsdk.apps.nxpimage_apps.nxpimage_signed_msg:Created Signed message Image:
Name:      Signed Message Image
Starts:    0x0
Ends:      0x23f
Size:      

## 3. Key Import Operation

### 3.1 Purpose

The key import operation allows secure importation of symmetric keys or private keys from asymmetric key pairs into the EdgeLock Secure Enclave key storage.

### 3.2 Key Features

- Encrypted Import: All imported keys must be encrypted (except specific cases like SRKH)
- Multiple Key Types: Supports AES, HMAC, ECC, and other key types
- Flexible Wrapping: Supports RFC3394 and AES-CBC wrapping algorithms
- Signed TLV Format: Uses signed TLV (Type-Length-Value) format for integrity

### 3.3 TLV Blob generation

Example of the configuration of TLV blob for importing the AES-256 key.

```yaml

command:
  KEY_IMPORT:
    # Key identification
    key_id: 0                               # Key store ID
    
    # Permitted algorithm
    permitted_algorithm: "ALL CIPHER"       # Hash algorithm for verification
    
    # Key attributes
    key_type: AES                           # Key type and hash
    key_size_bits: 256                      # Key size: 128/192/224/256/384/512
    key_lifetime: ELE_KEY_IMPORT_PERSISTENT # Persistence level
    key_lifecycle: OPEN_CLOSED              # Lifecycle restrictions
    key_usage:                              # Permission list
      - Encrypt
      - Decrypt
      
    # Wrapping configuration
    oem_mk_sk_key_id: 1                     # OEM_IMPORT_MK_SK key ID
    key_wrapping_algorithm: RFC3394         # RFC3394 or AES_CBC
    signing_algorithm: CMAC                 # Signature algorithm
    
    # Key data (two options)
    # Option 1: Pre-wrapped key and signature
    # wrapped_key: "0x00000000"
    # signature: "0x00000000000000000000000000000000"
    
    # Option 2: Raw key with master key (automatic wrapping)
    import_key: aes256.bin                  # Raw key file
    oem_import_mk_sk_key: "0xc7e7436ed876b0176d775a658c3e9bcf59c940a41a2673b0552b7d7df504d194" # Taken from the key exchange operation
    
    # Optional SRKH for salt derivation
    srkh: "0x00000000000000000000000000000000"
```

### 3.4 Supported Key Types

Key Type          | Value  | Supported Sizes (bits)
------------------|--------|------------------------
AES               | 0x2400 | 128, 192, 256
HMAC              | 0x1100 | 224, 256, 384, 512
Derived key       | 0x1200 | 256, 384
OEM_IMPORT_MK_SK  | 0x9200 | 128, 192, 256
ECC NIST          | 0x7112 | 128, 192, 256

### 3.5 Key Usage Permissions

Permission      | Value      | Description
----------------|------------|-------------
Cache           | 0x00000004 | Cache in secure memory (default)
Encrypt         | 0x00000100 | Encryption operations
Decrypt         | 0x00000200 | Decryption operations
Sign message    | 0x00000400 | Message signing (MAC/asymmetric)
Verify message  | 0x00000800 | Message verification
Sign hash       | 0x00001000 | Hash signing (asymmetric)
Verify hash     | 0x00002000 | Hash verification
Derive          | 0x00004000 | Key derivation operations



### 3.6 Export of TLV blob

```bash
nxpimage -v signed-msg tlv export -c inputs/tlv_import_aes256.yaml
```


In [4]:
YamlDiffWidget("inputs/tlv_import_aes256.diffc").html

nxpimage signed-msg tlv get-template -f mx93 -o workspace/tlv_import_aes256.yaml --force
Success. (TLV template: workspace/tlv_import_aes256.yaml created.)


#### 3.6.1 TLV blob - AES256

In [5]:
%! nxpimage -v signed-msg tlv export -c inputs/tlv_import_aes256.yaml

nxpimage -v signed-msg tlv export -c inputs/tlv_import_aes256.yaml
INFO:spsdk.image.ahab.tlv:The Import key Signed message created with raw key and OEM_IMPORT_MK_SK key.
INFO:spsdk.image.ahab.tlv:Derived OEM_IMPORT_WRAP_SK: f2fa66b83ed82acbe4dddb5c60966cabad61cf3116b542115f10f06cf6aee825
INFO:spsdk.image.ahab.tlv:Derived OEM_IMPORT_CMAC_SK: dc202a1160b4a5d11c088514e99e298b45aba12867543236fcd8774227f3e15c
INFO:spsdk.apps.nxpimage_apps.nxpimage_signed_msg:TLV blob size: 141 bytes
INFO:spsdk.apps.nxpimage_apps.nxpimage_signed_msg:TLV details:
TLV
  Key ID value: 0x00000000, 0
  Key import algorithm value: ALL CIPHER
  Key usage value: ['Encrypt', 'Decrypt']
  Key type value: AES
  Key bit size value: 0x00000100, 256
  Key life time value: ELE_KEY_IMPORT_PERSISTENT
  Key life cycle value: OPEN_CLOSED
  OEM Import MK SK key ID value: 0x00000001, 1
  Key wrapping algorithm: RFC3394
  Initial vector value: 00000000000000000000000000000000
  Key signing algorithm: CMAC
  Import key wrapped dat

#### 3.6.2 TLV blob - ECC384

In [6]:
YamlDiffWidget("inputs/tlv_import_ecc384.diffc").html

nxpimage signed-msg tlv get-template -f mx93 -o workspace/tlv_import_ecc384.yaml --force
Success. (TLV template: workspace/tlv_import_ecc384.yaml created.)


In [7]:
%! nxpimage -v signed-msg tlv export -c inputs/tlv_import_ecc384.yaml

nxpimage -v signed-msg tlv export -c inputs/tlv_import_ecc384.yaml
INFO:spsdk.image.ahab.tlv:The Import key Signed message created with raw key and OEM_IMPORT_MK_SK key.
INFO:spsdk.image.ahab.tlv:Derived OEM_IMPORT_WRAP_SK: f2fa66b83ed82acbe4dddb5c60966cabad61cf3116b542115f10f06cf6aee825
INFO:spsdk.image.ahab.tlv:Derived OEM_IMPORT_CMAC_SK: dc202a1160b4a5d11c088514e99e298b45aba12867543236fcd8774227f3e15c
INFO:spsdk.apps.nxpimage_apps.nxpimage_signed_msg:TLV blob size: 157 bytes
INFO:spsdk.apps.nxpimage_apps.nxpimage_signed_msg:TLV details:
TLV
  Key ID value: 0x00000000, 0
  Key import algorithm value: ECDSA SHA384
  Key usage value: ['Sign hash', 'Verify hash']
  Key type value: ECC NIST
  Key bit size value: 0x00000180, 384
  Key life time value: ELE_KEY_IMPORT_PERSISTENT
  Key life cycle value: OPEN_CLOSED
  OEM Import MK SK key ID value: 0x00000001, 1
  Key wrapping algorithm: RFC3394
  Initial vector value: 00000000000000000000000000000000
  Key signing algorithm: CMAC
  Import ke